In [9]:
import pickle
import numpy as np
import pandas as pd
import joblib
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

ImportError: cannot import name 'joblib' from 'sklearn.externals' (c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\externals\__init__.py)

In [4]:
#Read CSV, index+1 supaya index dimulai dari 1, tidak 0, memudahkan pembacaan dataframe dibawah.
df = pd.read_csv('diabetes.csv')
df.index += 1

In [ ]:
#melihat dataframe diabetes.csv
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,diabetes
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
#target / label. Kolom terakhir/kolom 'diabetes'.
#mengambil dataframe untuk disimpan menjadi array. 
#np_target = df.iloc[:,-1:].values #alternatif
np_target = df['diabetes'].values

In [ ]:
#Features. Mengambil kolom 1 - 8 dengan mengakses index. 
np_data = df.iloc[:,:-1].values

In [ ]:
#Naive Bayes
clf = GaussianNB()

In [ ]:
#Support Vector Classification model
#clf = svm.SVC(kernel='linear', gamma=0.001, C=100)

In [ ]:
#Split dataset, 20% testing 80% training.
X_train, X_test, y_train, y_test = train_test_split(np_data,np_target,test_size=0.2, random_state=None)

In [ ]:
X_train

array([[8.000e+00, 1.100e+02, 7.600e+01, ..., 2.780e+01, 2.370e-01,
        5.800e+01],
       [3.000e+00, 1.480e+02, 6.600e+01, ..., 3.250e+01, 2.560e-01,
        2.200e+01],
       [1.000e+00, 9.000e+01, 6.200e+01, ..., 2.510e+01, 1.268e+00,
        2.500e+01],
       ...,
       [0.000e+00, 1.020e+02, 5.200e+01, ..., 2.510e+01, 7.800e-02,
        2.100e+01],
       [2.000e+00, 1.120e+02, 7.800e+01, ..., 3.940e+01, 1.750e-01,
        2.400e+01],
       [5.000e+00, 1.300e+02, 8.200e+01, ..., 3.910e+01, 9.560e-01,
        3.700e+01]])

In [ ]:
#Cuma buat liat ukuran data train vs data test
len(X_train), len(X_test)

(614, 154)

In [ ]:
#Fit model SVM sesuai data training (proses training model)
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
#CrossValidation model yang sudah dibangun. Dengan parameter scoring=accuracy,precision, dan recall
scoring = ['accuracy', 'precision', 'recall']
scores = cross_validate(clf, X_test, y_test, scoring=scoring, cv=5, return_train_score=True)

In [ ]:
#Menampilkan hasil dari cross validation
scores

{'fit_time': array([0.00130391, 0.00078034, 0.00088215, 0.00098443, 0.00061941]),
 'score_time': array([0.0062263 , 0.00236559, 0.00470066, 0.00219297, 0.00205803]),
 'test_accuracy': array([0.625     , 0.83870968, 0.74193548, 0.76666667, 0.66666667]),
 'train_accuracy': array([0.77868852, 0.73170732, 0.76422764, 0.78225806, 0.76612903]),
 'test_precision': array([0.46153846, 0.8       , 0.63636364, 0.61538462, 0.5       ]),
 'train_precision': array([0.68292683, 0.60465116, 0.65853659, 0.69047619, 0.65909091]),
 'test_recall': array([0.54545455, 0.72727273, 0.63636364, 0.8       , 0.2       ]),
 'train_recall': array([0.66666667, 0.61904762, 0.64285714, 0.6744186 , 0.6744186 ])}

In [ ]:
#menampilkan test_accuracy dari cross validation, memiliki 5 value, tiap value adalah representasi akurasi dari tiap iterasi
scores['test_accuracy']

array([0.625     , 0.83870968, 0.74193548, 0.76666667, 0.66666667])

In [ ]:
#Hasil cross validation berupa dictionary. Kemudian dictionary disimpan menjadi dataframe. Hanya untuk keperluan pembacaan
df_scores = pd.DataFrame.from_dict(scores)
df_scores.index +=1
df_scores

,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall
1,0.001304,0.006226,0.625000,0.778689,0.461538,0.682927,0.545455,0.666667
2,0.000780,0.002366,0.838710,0.731707,0.800000,0.604651,0.727273,0.619048
3,0.000882,0.004701,0.741935,0.764228,0.636364,0.658537,0.636364,0.642857
4,0.000984,0.002193,0.766667,0.782258,0.615385,0.690476,0.800000,0.674419
5,0.000619,0.002058,0.666667,0.766129,0.500000,0.659091,0.200000,0.674419


In [ ]:
#DUMP MODEL
pickle.dump(clf, open('model_nb.pkl','wb'))

In [ ]:
sample = np.array([6,148,72,35,0,34,1,50])
clf.predict_proba(sample.reshape(1,-1))

array([[0.14397874, 0.85602126]])

In [ ]:
y_pred = clf.predict(np_data)
tn, fp, fn, tp = confusion_matrix(np_target, y_pred).ravel()

In [ ]:
acc = accuracy_score(np_target, y_pred)
prec= precision_score(np_target, y_pred)
recall = recall_score(np_target, y_pred)

print(acc, prec, recall)

0.76171875 0.6824034334763949 0.5932835820895522
